### Resources
Email with subject line "jarad@ quick stat(s) for the following? boards that support circuitpython
"
*****
From pt:

we're doing a post soon about the # of boards that support circuitpython that we sold in 2018, here are all the boards:
        https://github.com/adafruit/awesome-circuitpython#hardware

* Adafruit Circuit Playground Express
* Adafruit Feather M0 Express
* Adafruit Feather M4 Express
* Adafruit Feather nRF52840 Express
* Adafruit GEMMA M0
* Adafruit Express featuring the SAMD51
* Adafruit HalloWing M0 Express
* Adafruit ItsyBitsy M0
* Adafruit ItsyBitsy M4
* Adafruit METRO M0 Express
* Adafruit Metro M4
* Adafruit NeoTrellis M4
* Adafruit Trinket M0

(this list is incomplete) [jb]

there are also packs that include some of these, so we'll need to figure that as well.

i've cc'ed mike here who can assist and also scott who will be using this info once we have it.

lemme know if ya need more info to do this. the end result should be something like:

"In 2018 Adafruit sold over ###,#### boards that support CircuitPython" so it's just 1 number, a total of all them.

this is a rough estimate we do not need to be exact.

### Libraries

In [1]:
import sys
sys.path.insert(0, '/Users/jarad/Fake Folder/Python Libraries')

from jb_libraries import *
%matplotlib inline

### Settings

In [2]:
year = 2018

close_workbook = 'yes'

csv_path = '/Users/jarad/Fake Folder/InterDept/Ad Hoc/Parts That Support CircuitPython Report/CSVs/'

### Get parts

In [3]:
parts = pd.read_sql(
'''
SELECT

ptc.part_id,
pd.products_name,
ptc.categories_id,
cd.categories_name,
c.parent_id,
cd2.categories_name AS parent_name

FROM products_to_categories ptc
JOIN categories c ON ptc.categories_id = c.categories_id
JOIN products_description pd ON ptc.part_id = pd.part_id
JOIN categories_description cd ON ptc.categories_id = cd.categories_id
JOIN categories_description cd2 ON c.parent_id = cd2.categories_id
''', db)

col_fix(parts)

for col in parts.columns.tolist():
    try:
        parts[col] = parts[col].str.lower()
    except:
        pass

### Get skus

In [4]:
skus_main = pd.read_sql(
'''
SELECT
part_id,
sku_id
FROM skus
WHERE part_id > 0
''', db)

col_fix(skus_main)

df = pd.DataFrame(skus_main.groupby('part id')['sku id'].apply(list))
df.reset_index(inplace = True)

In [5]:
parts['sku ids'] = parts['part id'].map(dict(zip(df['part id'], df['sku id'])))

### Get products to stuff

In [6]:
pts_main = pd.read_sql(
'''
SELECT
*
FROM products_to_stuff
''', db)

col_fix(pts_main)

### Get CircuitPython boards
as per pt's email "jarad@ quick stat(s) for the following? boards that support circuitpython
"

In [7]:
cpb = parts[(parts['parent name'] == 'circuitpython')
           & (parts['categories name'] == 'boards')]

### Find what combos have these PNs

In [8]:
combos = pts_main[pts_main['contains part id'].isin(cpb['part id'])][['part id','contains part id']]

combos['products name'] = combos['part id'].map(dict(zip(parts['part id'], parts['products name'])))
combos['contains products name'] = combos['contains part id'].map(dict(zip(parts['part id'], parts['products name'])))
combos['part id and products name'] = 'PN ' + combos['part id'].map(str) + ': ' + combos['products name']

combo_parts = list(set(combos['part id']))

display(combos)

,part id,contains part id,products name,contains products name,part id and products name
7656,2680,2821,huzzah! adafruit.io internet of things feather esp8266,adafruit feather huzzah with esp8266 - loose headers,PN 2680: huzzah! adafruit.io internet of things feather esp8266
14441,3022,3046,paper signals bundle,assembled adafruit feather huzzah with esp8266 with headers,PN 3022: paper signals bundle
19333,3937,3333,adaptive design's morse code for gboard hardware interface pack,circuit playground express,PN 3937: adaptive design's morse code for gboard hardware interface pack
19919,3977,3517,getting started with circuit playground express book bundle,circuit playground express - base kit,PN 3977: getting started with circuit playground express book bundle
20202,4011,3333,circuit playground snow globe kit,circuit playground express,PN 4011: circuit playground snow globe kit
20273,4020,3938,adafruit neotrellis m4 with enclosure and buttons kit pack,adafruit neotrellis m4 mainboard - featuring samd51,PN 4020: adafruit neotrellis m4 with enclosure and buttons kit pack
20344,4025,3900,adafruit hallowing googly eyes bundle,adafruit hallowing m0 express,PN 4025: adafruit hallowing googly eyes bundle
20420,4028,3800,circuitpython starter kit with adafruit itsy bitsy m4,adafruit itsybitsy m4 express featuring atsamd51,PN 4028: circuitpython starter kit with adafruit itsy bitsy m4


### Make interim list of part_ids

In [9]:
all_pns = list(cpb['part id']) + combo_parts

### Find what parts have these skus

In [10]:
ls = []
for i in range(len(cpb)):
    skus = cpb.iloc[i]['sku ids']
    try:
        for s in skus:
            ls.append(s)
    except:
        pass
    
# find the part_ids for these skus    
a = skus_main[skus_main['sku id'].isin(ls)].copy()

# drop the dupe part_ids
a.drop_duplicates('part id', inplace = True)
a.reset_index(drop = True, inplace = True)

# drop where a part_id is in out cpb df
a.drop(a[a['part id'].isin(cpb['part id'])].index, inplace = True)
a.reset_index(drop = True, inplace = True)

if a.empty:
    print('move on, nothing here')
else:
    all_pns + list(a['part id'])
    display(a)

move on, nothing here


### Get sales data

In [11]:
pd.read_sql(
'''
SELECT
*
FROM orders_status
ORDER BY orders_status_id
''', db)

,orders_status_id,language_id,orders_status_name
0,1,1,Pending
1,2,1,Processing
2,3,1,Shipped
3,4,1,Update
4,5,1,Printed
5,6,1,Billed
6,7,1,Payment Received
7,8,1,Fraud - Pending
8,9,1,Fraud - Confirmed
9,10,1,Return


In [12]:
op_main = pd.read_sql(
'''
SELECT
DATE(date_purchased) AS date_purchased,
op.orders_id,
part_id,
products_quantity - products_quantity_free AS qty_bought,
products_quantity AS qty_total,
products_quantity_free AS qty_free
FROM orders_products op

JOIN orders o ON op.orders_id = o.orders_id
AND YEAR(date_purchased) = '''+ str(year) +'''
AND orders_status NOT IN (8,9,10,11,12,14,15)
AND payment_method != 'replacement order'

WHERE part_id IN '''+ str(tuple(all_pns)) +'''
''', db)

col_fix(op_main)

In [13]:
op_main['date purchased'] = pd.to_datetime(op_main['date purchased'])
op_main['year and month'] = jb_dates(op_main['date purchased'], 'year and month')

op_main['products name'] = op_main['part id'].map(dict(zip(parts['part id'], parts['products name'])))

for col in ['bought','free','total']:
    op_main['qty ' + col] = op_main['qty ' + col].map(int)

### Group by part

In [14]:
by_part = op_main.groupby(['part id','products name'])[['qty bought','qty free','qty total']].sum()

by_part['contained in'] = by_part.index.get_level_values(0).map(dict(zip(combos['contains part id'], combos['part id and products name'])))

### Merge to cpb df

In [15]:
final = pd.merge(cpb[['part id','products name']],
                by_part,
                how = 'left',
                on = ['part id','products name'])

final['contained in'].fillna(' ', inplace = True)

for col in ['bought','free','total']:
    final['qty ' + col].fillna(0, inplace = True)
    
final.sort_values('qty free', ascending = False, inplace = True)    

### Get totals

In [16]:
totals = pd.DataFrame(by_part.drop('contained in', 1).sum())
totals.columns = ['qty sum']

if totals.loc['qty bought'].item() + totals.loc['qty free'].item() != totals.loc['qty total'].item():
    raise ValueError('ur totals don\'t match')

totals.format_(['n0'])

,Qty Sum
qty bought,"145,254"
qty free,"10,936"
qty total,"156,190"


### Send to Excel

In [17]:
final_excel = final.copy()

for x in ['products name','contained in']:
    final_excel[x] = final_excel[x].str.title()

final_excel.columns = final_excel.columns.str.title()

ls = ['Total',
      '',
      by_part['qty bought'].sum().item(),
      by_part['qty free'].sum().item(),
      by_part['qty total'].sum().item(),
      '']

tot = pd.DataFrame(columns = final_excel.columns.tolist(), data = [ls])    
final_excel = pd.concat([final_excel,tot], sort = False)    

In [18]:
final_excel.format_([0,0,'n0','n0','n0',0])

,Part Id,Products Name,Qty Bought,Qty Free,Qty Total,Contained In
11,3333,Circuit Playground Express,"22,330","9,999","32,329",Pn 4011: Circuit Playground Snow Globe Kit
16,3500,Adafruit Trinket M0 - For Use With Circuitpython & Arduino Ide,"20,905",630,"21,535",
17,3501,Adafruit Gemma M0 - Miniature Wearable Electronic Platform,"6,498",276,"6,774",
8,3178,Adafruit Feather M0 With Rfm95 Lora Radio - 900Mhz,"5,290",20,"5,310",
6,3176,Adafruit Feather M0 Rfm69Hcw Packet Radio - 868 Or 915 Mhz,"3,367",6,"3,373",
0,2769,Circuit Playground Express Advanced Pack,"1,596",3,"1,599",
19,3517,Circuit Playground Express - Base Kit,"8,916",2,"8,918",Pn 3977: Getting Started With Circuit Playground Express Book Bundle
20,3727,Adafruit Itsybitsy M0 Express - For Circuitpython & Arduino Ide,"5,255",0,"5,255",
31,4064,Adafruit Grand Central M4 Express Featuring The Samd51,0,0,0,
30,4062,Adafruit Feather Nrf52840 Express,0,0,0,


In [20]:
if close_workbook == 'yes':
    
    writer = pd.ExcelWriter(csv_path + 'CircuitPython Boards Sold in 2018.xlsx')
    final_excel.to_excel(writer, sheet_name = 'Qty Sales by Part', index = False)
    writer.save()

In [ ]:
print('done')